In [2]:
import numpy as np
import pandas as pd

In [3]:
spamfile=pd.read_csv('spam.csv', encoding = "ISO-8859-1")

In [4]:
spamfile=spamfile[['v1','v2']]

In [5]:
stopword=pd.read_csv('stopwords.csv')

In [6]:
st=[]
with open('stopwords.txt','r') as stopwords:
    for i in stopwords:
        st.append(i.strip())
st

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'computer',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'do',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fify',
 'fill',
 'find',
 'fire',
 'first',
 'f

In [7]:
st1=list(stopword['words'])

In [8]:
st1

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'computer',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'do',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fify',
 'fill',
 'find',
 'fire',
 'first',
 'f

In [11]:
datalist=[]
for mail in spamfile.v2:
    sentence=[]
    sentlist=mail.strip().split('.')
    for sent in sentlist:
        wordlist=[]
        for word in sent.split():
            for words in word.split(','):
                if (words not in st) and len(words)!=0:
                    wordlist.append(words)
        if(len(wordlist)!=0):
            sentence.append(wordlist)
    datalist.append(sentence)

In [13]:
datalist1=[]
for mail in spamfile.v2:
    sentence=[]
    sentlist=mail.strip().split('.')
    for sent in sentlist:
        wordlist=[]
        for word in sent.split():
            for words in word.split(','):
                if (words not in st) and len(words)!=0:
                    sentence.append(words)
    datalist1.append(sentence)
#print(datalist1)

In [30]:
from collections import defaultdict
freqwords=defaultdict(int)
for mail in datalist1:
    for word in mail:
        freqwords[word]=freqwords[word]+1
#freqwords
abc=freqwords.copy()
for j in abc:
    if freqwords[j]==1:
        freqwords.pop(j)
#freqwords

In [126]:
filtdata=[]
for mail in datalist1:
    filt=[]
    for word in mail:
        if freqwords[word]>20:
            filt.append(word)
    filtdata.append(filt)
#print(filtdata)

import gensim

In [127]:
import gensim
dictbow=gensim.corpora.Dictionary(filtdata)


In [128]:
bowdata=[dictbow.doc2bow(mail) for mail in filtdata]
dense_bow = gensim.matutils.corpus2dense(bowdata, num_terms=len(dictbow))
dense_bow.shape

(460, 5572)

In [129]:
dense_bow=np.transpose(dense_bow)
#dense_bow
dense_bow.shape

(5572, 460)

In [130]:
train_set=[]
test_set=[]
x,z=dense_bow.shape
t=int(0.9*x)
for i in range(0,t):
    train_set.append(dense_bow[i])
for i in range(t,x):
    test_set.append(dense_bow[i])
#print(len(dense_bow))
print(len(train_set))
#print(len(test))
    

5014


In [131]:
target=[]
for i in spamfile.v1:
    if i == 'spam':
        target.append(0)
    else:
        target.append(1)
#pred

In [132]:
train_tag=target[:5014]
test_tag=target[5014:]

In [133]:
from sklearn import svm

In [134]:
classifier = svm.SVC()
classifier.fit(train_set,train_tag)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [135]:
predct=classifier.predict(test_set)

In [136]:
incorrectpred=(predct!=test_tag).sum()
incorrectpred

48

In [141]:
small=100
for too in range(1,100):
    print("ieration",too)
    filtdata=[]
    for mail in datalist1:
        filt=[]
        for word in mail:
            if freqwords[word]>too:
                filt.append(word)
        filtdata.append(filt)
    #print(filtdata)
    dictbow=gensim.corpora.Dictionary(filtdata)
    bowdata=[dictbow.doc2bow(mail) for mail in filtdata]
    dense_bow = gensim.matutils.corpus2dense(bowdata, num_terms=len(dictbow)).transpose()
    #dense_bow.shape
    train_set=[]
    test_set=[]
    x,z=dense_bow.shape
    t=int(0.9*x)
    for i in range(0,t):
        train_set.append(dense_bow[i])
    for i in range(t,x):
        test_set.append(dense_bow[i])
    #print(len(dense_bow))
    #print(len(train_set))
    #print(len(test))
    target=[]
    for i in spamfile.v1:
        if i == 'spam':
            target.append(0)
        else:
            target.append(1)
    #pred
    train_tag=target[:5014]
    test_tag=target[5014:]
    classifier = svm.SVC()
    classifier.fit(train_set,train_tag)
    predct=classifier.predict(test_set)
    incorrectpred=(predct!=test_tag).sum()
    #print(incorrectpred)
    if incorrectpred<small:
        temp=too
        small=incorrectpred
print(temp)
print(incorrectpred)

ieration 0
ieration 1
ieration 2
ieration 3
ieration 4
ieration 5
ieration 6
ieration 7
ieration 8
ieration 9
ieration 10
ieration 11
ieration 12
ieration 13
ieration 14
ieration 15
ieration 16
ieration 17
ieration 18
ieration 19
ieration 20
ieration 21
ieration 22
ieration 23
ieration 24
ieration 25
ieration 26
ieration 27
ieration 28
ieration 29
ieration 30
ieration 31
ieration 32
ieration 33
ieration 34
ieration 35
ieration 36
ieration 37
ieration 38
ieration 39
ieration 40
ieration 41
ieration 42
ieration 43
ieration 44
ieration 45
ieration 46
ieration 47
ieration 48
ieration 49
ieration 50
ieration 51
ieration 52
ieration 53
ieration 54
ieration 55
ieration 56
ieration 57
ieration 58
ieration 59
ieration 60
ieration 61
ieration 62
ieration 63
ieration 64
ieration 65
ieration 66
ieration 67
ieration 68
ieration 69
ieration 70
ieration 71
ieration 72
ieration 73
ieration 74
ieration 75
ieration 76
ieration 77
ieration 78
ieration 79
ieration 80
ieration 81
ieration 82
ieration 83
ie